# Sentence Transformers 학습

REFERENCE : https://colab.research.google.com/github/Huffon/klue-transformers-tutorial/blob/master/sentence_transformers.ipynb#scrollTo=d9598bd1-bf9a-4abc-bf0a-d79f0b0f45a9

참고 사이트는 klue/roberta-base 모델을 KLUE내 STS데이터 셋을 활용해 모델을 훈련하는 예제를 다룬다 
학습을 통해 얻어질 sentence-klue-robert-base모델은 입력된 문장의 임베딩을 계산해 유사도를 예측하는데 사용할 수 있다.
학습 과정 이후에는 간단한 예제 코드를 통해 모델이 어떻게 활용되는지 실습할 예정이다.
모든 소스코드는 sentence-transformers원 라이브러리를 참고하였다.

먼저 , 노트북을 실행하는데 필요한 라이브러리를 설치 후 모델 훈련을 위해서는 setence-transformers가 , 학습로드를 위해서는 datasets라이브러리의 설치가 필요하다

In [ ]:
! pip install sentence-transformers
! pip install sentence-transformers datasets

# Sentence Transformers 학습

노트북을 실행하는데 필요한 라이브러리들을 모두 임포트한다.

In [ ]:
import math
import logging
from datetime import datetime

import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
from sentence_transformers import SentenceTransformer,LoggingHandler,losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample

학습경과를 지켜보는데 사용될 logger를 초기화한다.
학습에 필요한 정보를 변수로 기록한다.

In [ ]:
logging.basicConfig(
    format="%(asctime)s-%(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    level=logging.INFO,
    handlers=[LoggingHandler()]
)

본 노트북에서는 klue-roberta-base모델을 활용하지만 , https://huggingface.co/klue 페이지에서 더 다양한 사전학습 언어 모델을 확인 할 수 있음

In [ ]:
model_name="klue/roberta-base"

모델 정보 외에도 학습에 필요한 하이퍼파리미터를 정의한다.

In [ ]:
train_batch_size=32
num_epochs=4
model_save_path="output/training_klue_"+model_name.replace("/","-")+"-"+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

앞에서 정의한 사전학습 언어 모델을 로드한다. 
sentence-transformers는 huggingFace의 transformers와 호환이 잘 이루어지고 있기 때문에, 모델 허브에 올라와있는 대부분의 언어 모델을 임베딩 추출할 Embedder로 사용할 수 있다.

In [ ]:
embedding_model=models.Transformer(model_name)

Embedder에서 추출된 토큰 단위 임베딩을 가지고 문장 임베딩을 어떻게 계산할 것인지를 결정하는 Pooler를 정의한다.
여러 Pooling기법이 있지만, 예제 노트북에서는 MeanPooling을 사용한다.
MeanPooling이란 모델이 반환한 모든 토큰 임베딩을 더해준 후, 더해진 토큰 개수만큼 나누어 문장을 대표하는 임베딩으로 사용하는 기법을 의미한다.

In [ ]:
pooler=models.Pooling(
    embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False
)

Embedder와 Pooler를 정의했으므로, 이 두 모듈로 구성된 하나의 모델을 정의한다.
modules에 입력으로 들어가는 모듈이 순차적으로 임베딩 과정에 사용이 된다고 생각하면 된다.

In [ ]:
model=SentenceTransformer(modules=[embedding_model,pooler])

이제 학습에 사용될 KLUE STS데이터 셋을 다운로드 및 로드한다.

In [ ]:
datasets=load_dataset("klue","sts")

In [ ]:
datasets

이제 테스트에 활용할 데이터를 얻어야 할 차례이다. 위에서 살펴본바와 같이 KLUE내 STS데이터 셋은 테스트 데이터셋을 포함하고 잇지 않는다. 따라서 실습을 원활한 진행을 위해 다른 벤티 마크 STS 데이터 셋인 KorSTS데이터셋을 다운로드및 로드하여 사용하도록 하겠다.


In [ ]:
testsets=load_dataset("kor_nlu","sts")

In [ ]:
testsets

In [ ]:
testsets['train'][0]

이제 앞서 얻어진 데이터 셋은 sentence-transformers훈련 양식에 맞게 변환해주는 작업을 거쳐야 한다. 
두 데이터 모두 0\~5점 사이값으로 유사도가 기록되어 있기 때문에 0.0\~1.0 스케일로 정규화를 시켜주는 작업을 거치게된다. 

In [ ]:
train_samples=[]
test_samples=[]
dev_samples=[]

#훈련, 검증 데이터 예제 변환
for phase in ["train","validation"]:
  examples=datasets[phase]

  for example in examples:
    score=float(example['labels']['label'])/5.0
    input_example=InputExample(
        texts=[example['sentence1'],example['sentence2']],
        label=score
    )
    if phase == "validation":
      dev_samples.append(input_example)
    else:
      train_samples.append(input_example)

#KorSTS내 테스트 데이터 예제 반환
for example in testsets["test"]:
  score=float(example["score"])/5.0
  if example["sentence1"]and example["sentence2"]:
    input_example=InputExample(
        texts=[example["sentence1"],example["sentence2"]],
        label=score
    )
  test_samples.append(input_example)

In [ ]:
train_samples[0].texts,train_samples[0].label

이제 학습에 사용될 DataLoader와 Loss를 설정해주도록 합니다.
CosineSimilarityLoss는 입력된 두 문자으이 임베딩 간 코사인 유사도와 골드 라벨 간 차이를 통해 계산되게 한다.

In [ ]:
train_dataloader=DataLoader(
    train_samples,
    shuffle=True,
    batch_size=train_batch_size
)
train_loss=losses.CosineSimilarityLoss(model=model)

모델 검증에 사용될 Evaluator를 정의한다.
앞서 얻어진 검증데이터를 활용하여, 모델의 문장 임베딩 간 코사인 유사도가 얼마나 골드 라벨에 가까운지 계산하는 역할을 수행함

In [ ]:
evaluator=EmbeddingSimilarityEvaluator.from_input_examples(
    dev_samples,
    name="sts-dev"
)

모델 학습에 사용될 Warm up Steps를 설정한다.
다양한 방법으로 스텝 수를 결정할 수 있겠지만, 예제 노트북에서는 원 예제 코들 따라 훈련 배치 수의 10%만큼으로 값을 설정한다.

In [ ]:
warmup_steps=math.ceil(len(train_dataloader)*num_epochs*0.1)
logging.info(f"Warmup-Steps:{wrarmup_steps}")

이제 앞서 얻어진 객체, 값들을 가지고 모델 훈련을 진행한다. 
setnece-transformers에서는 다음과 같이 fit함수를 사용해 모델을 훈련과 검증이 가능하다.
훈련과정을 통해 매 에폭마다 얻어지는 체크포인트에 대해 Evaluator가 학습된 모델의 코사인 유사도와 골드 라벨 간 피어슨,스피어만 상관계수를 계산해 기록을 남기게 된다.

In [ ]:
model.fit(
    train_objectives=[(train_dataloader,train_loss)],
    evaluator=evaluator,
    epochs=num_epochs,
    evaluation_steps=1000,
    warmup_steps=warmup_steps,
    output_path=model_save_path
)

In [ ]:
print(len(train_dataloader))